In [2]:
"""Automating the verification processes on loans for financial auditing, the idea is to input each loan data once then the dataframe computes interests, capital..."""

# Creating a dataframe here as an example of the input data, real input will come from an excel spreadsheet
# By the way, this whole project can be used directly in excel thanks to the =PY() function
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

clot = pd.to_datetime("31/12/2024", dayfirst=True)
today = pd.to_datetime("25/01/2025", dayfirst=True)

compte = [51200, 51201, 51202]
IdPret = [1, 2, 3]
Type = ["InFine", "AmortissableLin", "Amortissable"]
DateDebut = ["20/04/2020", "31/12/2023", "01/01/2010"]
DateDebut = pd.to_datetime(DateDebut, dayfirst=True)
DateEcheance = ["20/04/2027", "31/12/2026", "01/01/2030"]
DateEcheance = pd.to_datetime(DateEcheance, dayfirst=True)
Interet = [0.03, 0.02, 0.05]
Montant = [1000000, 2000000, 50000]
df = pd.DataFrame({"Compte":compte, "IdPret":IdPret, "Type":Type, "DateDebut":DateDebut, "DateEcheance":DateEcheance, "Int":Interet, "Montant":Montant})
df

,Compte,IdPret,Type,DateDebut,DateEcheance,Int,Montant
0,51200,1,InFine,2020-04-20,2027-04-20,0.03,1000000
1,51201,2,AmortissableLin,2023-12-31,2026-12-31,0.02,2000000
2,51202,3,Amortissable,2010-01-01,2030-01-01,0.05,50000


In [3]:
# Interest
df["IntEx"] = df['Montant'] * df['Int']   # je crois qu' elle ne sert à rien si elle n'est pas contreproductive

df['Duree'] = (df['DateEcheance'] - df['DateDebut']) / pd.Timedelta(days=365.25)

df["Frequence"] = "Trimestrielle"
print(df)

   Compte  IdPret             Type  DateDebut DateEcheance   Int  Montant  \
0   51200       1           InFine 2020-04-20   2027-04-20  0.03  1000000   
1   51201       2  AmortissableLin 2023-12-31   2026-12-31  0.02  2000000   
2   51202       3     Amortissable 2010-01-01   2030-01-01  0.05    50000   

     IntEx      Duree      Frequence  
0  30000.0   6.997947  Trimestrielle  
1  40000.0   3.000684  Trimestrielle  
2   2500.0  20.000000  Trimestrielle  


In [4]:
# Corresponding number of month for each period
def Frequence_A_Num(row):
    if row['Frequence'] == "Trimestrielle":
        return 4
    elif row['Frequence'] == "Mensuelle":
        return 12
    elif row['Frequence'] == "Annuelle":
        return 1
    elif row['Frequence'] == "Semestrielle":
        return 2
    else:
        raise ValueError("Fréquence non reconnue")

df['FrequenceN'] = df.apply(Frequence_A_Num, axis=1)

# Number of repayments since inception of loan
def Nb_Ech_Depuis_Debut(row):
    delta = relativedelta(clot, row['DateDebut']) # This function takes into account the fact that dates can be mid-month
    return (delta.years * 12 + delta.months) / (12 / row['FrequenceN'])
df['NbEchDepuisDebut'] = df.apply(Nb_Ech_Depuis_Debut, axis=1)

print(df)

   Compte  IdPret             Type  DateDebut DateEcheance   Int  Montant  \
0   51200       1           InFine 2020-04-20   2027-04-20  0.03  1000000   
1   51201       2  AmortissableLin 2023-12-31   2026-12-31  0.02  2000000   
2   51202       3     Amortissable 2010-01-01   2030-01-01  0.05    50000   

     IntEx      Duree      Frequence  FrequenceN  NbEchDepuisDebut  
0  30000.0   6.997947  Trimestrielle           4         18.666667  
1  40000.0   3.000684  Trimestrielle           4          4.000000  
2   2500.0  20.000000  Trimestrielle           4         59.666667  


In [14]:
# Payment during the exercice, differenciated by loan type

def CRD(row):
    # InFine
    if row['Type'] == 'InFine':
        if row['DateEcheance'] > clot :
            return row['Montant']
        else:
            return 0
    # Amortissable constant (linéaire)
    elif row['Type'] == 'AmortissableLin':
        Annuite = (row['Montant'] / (row['Duree'] * row['FrequenceN'])) # Remboursement du capital sur une échéance
        return row['Montant'] - Annuite * row['NbEchDepuisDebut']

    # Echeance constante
    elif row['Type'] == 'Amortissable':
        TauxParPeriode = row['Int'] / row['FrequenceN'] # Taux périodique actualisé en fonction du nombre d'échéance sur une année
        NombreEch = row['Duree'] * row['FrequenceN'] # Nombre total d'échéance de l'emprunt
        AnnuiteConstante = row['Montant'] * (TauxParPeriode / (1 - (1 +TauxParPeriode)**-NombreEch)) #Annuité constante
        
        crd = row['Montant']
        for i in range(int(row['NbEchDepuisDebut'])):
            interet = crd * TauxParPeriode
            capital = AnnuiteConstante - interet
            crd -= capital
            
        return crd
        
        
        
df['CRD'] = df.apply(CRD, axis=1)



print(df)

   Compte  IdPret             Type  DateDebut DateEcheance   Int  Montant  \
0   51200       1           InFine 2020-04-20   2027-04-20  0.03  1000000   
1   51201       2  AmortissableLin 2023-12-31   2026-12-31  0.02  2000000   
2   51202       3     Amortissable 2010-01-01   2030-01-01  0.05    50000   

     IntEx      Duree      Frequence  FrequenceN  NbEchDepuisDebut  \
0  30000.0   6.997947  Trimestrielle           4         18.666667   
1  40000.0   3.000684  Trimestrielle           4          4.000000   
2   2500.0  20.000000  Trimestrielle           4         59.666667   

            CRD  
0  1.000000e+06  
1  1.333485e+06  
2  1.822873e+04  


In [ ]:
#will be removed later on
# Remaining principal (CRD) : while the due date > to exercice date (clot) the value is set to loan amount, else it's zero
df["CRD"] = np.where(
    (df['Type'] == 'InFine') & (df['DateEcheance'] > clot),
    df['Montant'],
    0 )

df["CRD"] = np.where(
    df['Type'] == 'AmortissableLin',
    df['Montant'] / ((df['DateEcheance'].dt.year - df['DateDebut'].dt.year) * df['FrequenceN']) * df['NbEchDepuisDebut'], #Montant par échéance * nombre d'échéance passées
    0
)